# 뉴스 빅데이터 분석 데이터 추출 & 사용자 사전 추가

In [ ]:
def Extract_Keyword_from_news_bigdata(path):
    from glob import glob
    fileList = glob(path+'*.csv')
    listIs =list()
    for i in fileList:
        df = pd.read_csv(i,engine='python')
        f = list(filter(lambda x: x[1] in ['중요키워드','키워드','keword','토픽키워드','토픽 키워드'], enumerate(df.columns.values)))
        if not len(f) == 0:
            listIs += list(df[df.columns[f[0][0]]].values)
    outlist = []
    for ix in list(filter(lambda x: type(x)!=float, listIs)):
        outlist += ix.split(',')
    outlist1 = list(map(lambda x: ''.join(x.split('_')), outlist))
    outlist2 = list(map(lambda x: ' '.join(x.split('_')), outlist))
    return outlist1+outlist2

In [ ]:
dbpath = 'C:/Users/pc/Anaconda3/lib/site-packages/ckonlpy/data/twitter/noun/'

In [ ]:
path1 = './data/nnews/high_frequency_noun/'
path2 = './data/news/mainissue/'
path3 = './data/news/newstopic/'
path4 = './data/news/people/'
path5 = './data/news/4th_industry/'
path6 = './data/news/have_negative_positive/constitution/'
path7 = './data/news/have_negative_positive/household_debt/'
path8 = './data/news/have_negative_positive/olymphic/'
path9 = './data/news/have_negative_positive/'
path10 = './data/news/'

pathlist = [path1, path2, path3,
            path4, path5, path6,
            path7, path8, path9,
           path10]
if not os.path.isfile(dbpath+'from_news.txt'):
    with open(dbpath+'from_news.txt','w') as f:
        for path in pathlist:
            out = Extract_Keyword_from_news_bigdata(path)
            f.write('\n'.join(out))

In [1]:
import pandas as pd 
from ckonlpy.tag import Twitter as ctwitter
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
import numpy as np
import matplotlib.pyplot as plt
from konlpy.tag import Mecab, Twitter
from konlpy.utils import pprint
import os
def Stopwords(file):
    stopwords = open(file,'r',  encoding='utf-8').readlines()
    stopwords = list(map(lambda x:x.strip(), stopwords))
    return stopwords
stopwords = Stopwords('./data/koreanStopwords.txt') +Stopwords('./data/newspress.txt')

In [2]:
mecab = Mecab()
ct = ctwitter()
ot = Twitter()
xxxx = ct._dictionary._pos2words
list(map(lambda x: len(xxxx[x]), xxxx))

[512, 5068471, 5532]

# 자연어 처리

# 분석

In [3]:
# -*- coding: utf-8 -*-
import sys
sys.path.append('/Users/hyunyoun/Documents/GitHub/Private_Project/personal_project/')
from collections import defaultdict
from datetime import datetime, timedelta
import time, re, pickle, itertools
import chat_bot as cb
import Database_Handler as dh
from multiprocessing import Pool
import urllib3, json
import nltk
from __future__ import print_function, unicode_literals
from gensim.models.word2vec import Word2Vec
from sklearn.preprocessing import normalize
import numpy as np
from operator import itemgetter
import os

def OpenAPI():
    import pickle
    openapi = pickle.load(open('./etri_open_api_access_key', 'rb'))
    return openapi
def USE_ETRI_ANALYSIS(analysisCode, text):    
    openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU"
    accessKey = OpenAPI().strip()
    requestJson = {
        'access_key' : accessKey,
        'argument' : {
            'text' : text,
            'analysis_code' : analysisCode
        }
    }
    http = urllib3.PoolManager()
    response = http.request(
        "POST",
        openApiURL,
        headers={"Content-Type": "application/json; charset=UTF-8"},
        body=json.dumps(requestJson)
    )
    return json.loads(str(response.data,'utf-8')), str(response.status)

def USE_ETRI_INFO(fWord, sWord):
    openApiURL = "http://aiopen.etri.re.kr:8000/WiseWWN/WordRel"
    accessKey = OpenAPI().strip()
    firstWord = fWord
    #firstSenseId = fSenseId
    secondWord = sWord
    #secondSenseId = sSenseId
    requestJson = {
        "access_key": accessKey,
        "argument": {
        'first_word': firstWord,
        #'first_sense_id': firstSenseId,
        'second_word': secondWord,
        #'second_sense_id': secondSenseId
        }}
    http = urllib3.PoolManager()
    response = http.request("POST",openApiURL,headers={"Content-Type": "application/json; charset=UTF-8"},
    body=json.dumps(requestJson))
    return json.loads(str(response.data, 'utf-8')), str(response.status)

def USE_ETRI_RELATION(word):
    openApiURL = "http://aiopen.etri.re.kr:8000/WiseWWN/Word"
    accessKey = OpenAPI().strip()
    word = word
    requestJson = {
    "access_key": accessKey,
    "argument": {"word": word}}
    http = urllib3.PoolManager()
    response = http.request("POST",openApiURL,headers={"Content-Type": "application/json; charset=UTF-8"},
                            body=json.dumps(requestJson))
    return json.loads(str(response.data, 'utf-8')), str(response.status)

def Extract_Text_Info(idx):
    morp = list(map(lambda x: (x['lemma'], x['type']),idx['morp']))
    wsd = list(map(lambda x: (x['text'], x['type']),idx['WSD']))
    word = list(map(lambda x: (x['text'], x['type']),idx['word']))
    ne = list(map(lambda x: (x['text'], x['type']),idx['NE']))
    return morp, wsd, word, ne

def Run_ETRI_Analysis(stopwordsList, text):
    etri = USE_ETRI_ANALYSIS('srl', text)['return_object']    
    etri = etri['sentence']
    morp = [] ; wsd = [] ; word = [] ; ne = [] ; srl = [] ; dependency = []
    for i in etri:
        x = Extract_Text_Info(i)
        morp += x[0]
        wsd += x[1]
        word +=x[2]
        ne += x[3]
        for ii in i['SRL']:
            srl += list(map(lambda x: (x['text'], x['type']),ii['argument']))
        dependency += list(map(lambda x: (x['text'], x['label']), i['dependency']))
    return morp, wsd, word, ne, srl, dependency, etri
    
if __name__ == '__main__':
    pass

C:\Users\pc\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [4]:
#site = 'Naver'
site = 'daum'
if site == 'daum':
    collection = 'newsDaum'
    etri_outcome = './data/ETRI_OUTCOME/daum2/'
elif site.lower() == 'naver':
    collection = 'newsNaver_copy'
    etri_outcome = './data/ETRI_OUTCOME/naver2/'

In [5]:
mongodb = dh.ToMongoDB(*dh.AWS_MongoDB_Information())
dbname = 'hy_db'
useDb = dh.Use_Database(mongodb, dbname)
slack = cb.Slacker(cb.slacktoken())
useCollection = dh.Use_Collection(useDb, collection)

In [6]:
keywordList = list()
pressList = list()
for data in useCollection.find({'site':site}):
    pressList.append(data['press'])
    
    if len(data['mainText'].strip()) !=0:
        if site=='daum':
            if data['keywords'] !='NaN':
                keywordList += list(map(lambda x: ''.join(x.split(' · ')[-1].split(' ')), data['keywords']))
        else:
            pass
        idIs = data['_id']._ObjectId__id
        if not os.path.isfile(etri_outcome+idIs.hex()+'.picked.txt'):
            #print (idIs.hex())
            try:
                etri = USE_ETRI_ANALYSIS('srl', data['mainText'])
            except:
                print (etri[1])
                break
            else:
                pickle.dump(etri[0], open(etri_outcome+idIs.hex()+'.picked.txt','wb'))
    break

In [7]:
testdata = pickle.load(open(etri_outcome+idIs.hex()+'.picked.txt','rb'))

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize as sknorm

class SentenceTokenizer(object):
    def __init__(self):
        self.twitter = ct
        #self.twitter = ot
        self.stopwords = ['중인' ,'만큼', '마찬가지', '꼬집었', "연합뉴스", "데일리", "동아일보", "중앙일보", "조선일보", "기자"
        ,"아", "휴", "아이구", "아이쿠", "아이고", "어", "나", "우리", "저희", "따라", "의해", "을", "를", "에", "의", "가",]    
    def text2sentences(self, text):
        sentences = re.sub('\. ','.\n', text).split('\n')
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
        return sentences
    def get_nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            if sentence is not '':
                nouns.append(' '.join([noun for noun in self.twitter.phrases(str(sentence))
                                                    if noun not in self.stopwords and len(noun) > 1]))
                nouns.append(' '.join([noun for noun in self.twitter.nouns(str(sentence))
                                                    if noun not in self.stopwords and len(noun) > 1]))
        return nouns
    
class GraphMatrix(object):
    def __init__(self):
        self.tfidf = TfidfVectorizer()
        self.cnt_vec = CountVectorizer()
        self.graph_sentence = []
    def build_sent_graph(self, sentence):
        tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        return self.graph_sentence
    def build_words_graph(self, sentence):
        cnt_vec_mat = sknorm(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = self.cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}

class Rank(object):
    def get_ranks(self, graph, d=0.82): # d = damping factor
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0 # diagonal 부분을 0으로
            link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
            if link_sum != 0:
                A[:, id] /= link_sum
            A[:, id] *= -d
            A[id, id] = 1
        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
        return {idx: r[0] for idx, r in enumerate(ranks)}

class TextRank(object):
    def __init__(self, text):
        self.sent_tokenize = SentenceTokenizer()
        self.sentences = self.sent_tokenize.text2sentences(text)
        self.nouns = self.sent_tokenize.get_nouns(self.sentences)
        self.graph_matrix = GraphMatrix()
        self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
        self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)
        self.rank = Rank()
        self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
        self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)
        self.word_rank_idx = self.rank.get_ranks(self.words_graph)
        self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)
    def summarize(self, sent_num=3):
        summary = []
        index=[]
        for idx in self.sorted_sent_rank_idx[:sent_num]:
            index.append(idx)
        index.sort()
        for idx in index:
            summary.append(self.sentences[idx])
        return summary
    def keywords(self, word_num=7):
        rank = Rank()
        rank_idx = rank.get_ranks(self.words_graph)
        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)
        keywords = []
        index=[]
        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)
        #index.sort()
        for idx in index:
            keywords.append(self.idx2word[idx])
        return keywords
textrank = TextRank(data['mainText'])
for row in textrank.summarize(3):
    print(row)
    print()
print('keywords :',textrank.keywords())

이날은 특집 명에 맞게 녹화 전까지 철저히 출연자들이 서로 이름을 공유하지 못하도록 조치를 취했다.

출연자들은 서로의 이름을 추측하며 자신의 이름보다 특이 한 지 궁금해 했다.

이날 8명의 출연자는 모두 "출석 부르는 시간, 자기소개 시간을 가장 싫어한다"며 다른 이름이지만 같은 심정으로 공감 대화를 나눴다.

keywords : ['이름', '한명회', '출연자', '방송', '이날', '시간', '이름을']


In [9]:
from krwordrank.word import KRWordRank
from krwordrank.hangle import normalize
def RunWordKRwordRank(text, min_count):
    #min_count = 2   # 단어의 최소 출현 빈도수 (그래프 생성 시)
    max_length = 10 # 단어의 최대 길이
    wordrank_extractor = KRWordRank(min_count, max_length)
    beta = 0.85   # PageRank의 decaying factor beta
    max_iter = 100
    verbose = False
    textIs = re.sub('\. ','.\n', text).split('\n')
    textIs = list(map(lambda x: normalize(x, english = True, number = True), textIs))
    keywords, rank, graph = wordrank_extractor.extract(textIs, beta, max_iter, verbose)
    return keywords, rank, graph

In [10]:
workrank1 = RunWordKRwordRank(data['mainText'], 3)
workrank1[0]

done


{'방송': 1.360025084375334,
 '시간': 1.3916619125260357,
 '이름': 1.6555387386756775,
 '출연자': 0.816209243859062,
 '한명회': 1.9429708517486093}

In [11]:
from lexrankr import LexRank
lexrank = LexRank()

In [12]:
lexrank.summarize(data['mainText'])

In [13]:
lexrank.probe(None)

["영상 바로보기 [마이데일리 = 이승길 기자] JTBC '내 이름을 불러줘-한명(名)회'(이하 '한명회')가 막을 내린다",
 '이날은 특집 명에 맞게 녹화 전까지 철저히 출연자들이 서로 이름을 공유하지 못하도록 조치를 취했다']

In [14]:
y = lexrank.probe(k=3)
y

["영상 바로보기 [마이데일리 = 이승길 기자] JTBC '내 이름을 불러줘-한명(名)회'(이하 '한명회')가 막을 내린다",
 '이날은 특집 명에 맞게 녹화 전까지 철저히 출연자들이 서로 이름을 공유하지 못하도록 조치를 취했다',
 '그중 유독 눈에 띄는 한 남성 출연자의 기상천외한 이름이 밝혀져, 나머지 출연진은 물론 MC들도 당황하며 이름을 재차 물어 웃음을 자아냈다']

In [ ]:
'.\n'.join(y)

In [17]:
ot.pos('.\n'.join(y))

[('영상', 'Noun'),
 ('바로', 'Noun'),
 ('보기', 'Noun'),
 ('[', 'Punctuation'),
 ('마이데일리', 'Noun'),
 ('=', 'Punctuation'),
 ('이승', 'Noun'),
 ('길', 'Noun'),
 ('기자', 'Noun'),
 (']', 'Punctuation'),
 ('JTBC', 'Alpha'),
 ("'", 'Punctuation'),
 ('내', 'Noun'),
 ('이름', 'Noun'),
 ('을', 'Josa'),
 ('불러', 'Verb'),
 ('줘', 'Eomi'),
 ('-', 'Punctuation'),
 ('한', 'Determiner'),
 ('명', 'Noun'),
 ('(', 'Punctuation'),
 ('名', 'Foreign'),
 (')', 'Punctuation'),
 ('회', 'Noun'),
 ("'(", 'Punctuation'),
 ('이하', 'Noun'),
 ("'", 'Punctuation'),
 ('한명회', 'Noun'),
 ("')", 'Punctuation'),
 ('가', 'Verb'),
 ('막', 'Noun'),
 ('을', 'Josa'),
 ('내', 'Determiner'),
 ('린다', 'Noun'),
 ('.', 'Punctuation'),
 ('이', 'Determiner'),
 ('날', 'Noun'),
 ('은', 'Josa'),
 ('특집', 'Noun'),
 ('명', 'Noun'),
 ('에', 'Josa'),
 ('맞게', 'Verb'),
 ('녹화', 'Noun'),
 ('전', 'Noun'),
 ('까지', 'Josa'),
 ('철저히', 'Adjective'),
 ('출연자', 'Noun'),
 ('들', 'Suffix'),
 ('이', 'Josa'),
 ('서로', 'Noun'),
 ('이름', 'Noun'),
 ('을', 'Josa'),
 ('공유', 'Noun'),
 ('하지', 'Verb')

In [20]:
testData = pickle.load(open(etri_outcome+idIs.hex()+'.picked.txt','rb'))

In [21]:
etri = testData['return_object']['sentence']
morp = [] ; wsd = [] ; word = [] ; ne = [] ; srl = [] ; dependency = []

for i in etri:
    x = Extract_Text_Info(i)
    morp += x[0]
    wsd += x[1]
    word +=x[2]
    ne += x[3]
    for ii in i['SRL']:
        srl += list(map(lambda x: (x['text'], x['type']),ii['argument']))
    dependency += list(map(lambda x: (x['text'], x['label']), i['dependency']))
    

* 형태소 분석 : morp
* 어휘의미 분석 : wsd (동음이의어)
* 어휘의미 분석 : wsd_poly (다의어)
* 개체명 인식 : ner
* 의존구문 분석 : dependency
* 의미역 인식 : srl

In [22]:
morp1 = list(map(lambda x: x[0], morp))
wsd1 = list(map(lambda x: x[0], wsd))
word1 = list(map(lambda x: x[0], word))
ne1 = list(map(lambda x: x[0], ne))
srl1 = list(map(lambda x: x[0], srl))
dependency1 = list(map(lambda x: x[0], dependency))
outlist = []
for idx in range(len(etri)):
    y = pd.DataFrame(etri[idx]['dependency'])
    y2 = y[y['mod'].apply(lambda x: len(x)) ==y['mod'].apply(lambda x: len(x)).max()]
    y3 = y2[y2['weight'] == y2['weight'].max()]
    y4 = y[y['id'].isin(y3['mod'][y3['mod'].index[0]])]
    y5 = y[y['weight'] ==y4['weight'].max()]
    y6 = y2.text.values.tolist() + y5.text.values.tolist()
    y7 = y[y.text.isin(y6)]    
    out = ' '.join(y7.text.tolist())
    outlist.append(out)

In [32]:
vect = TfidfVectorizer(tokenizer=ct.phrases, ngram_range=(1, 4),
                    stop_words=stopwords)
vect.fit(morp1)
vect.fit(wsd1)
vect.fit(word1)
vect.fit(ne1)
vect.fit(srl1)
vect.fit(outlist)
count = vect.transform(re.sub('\. ','.\n', data['mainText']).split('\n')).toarray()
count1 = count.sum(axis = 0)
idx = np.argsort(-count1)
count1 = count1[idx]
feature_name = np.array(vect.get_feature_names())[idx]
out = dict(zip(feature_name, count1))
sorted(out.items(), key=lambda x: x[1], reverse=True)

[('이름', 3.7396725708214373),
 ('한명회', 2.3645921404341106),
 ('방송', 1.4051585357702066),
 ('서로', 1.2071067811865475),
 ('녹화', 1.0),
 ('내린다', 0.9136254000951356),
 ('jtbc', 0.81298699525717766),
 ('jtbc 제공', 0.70710678118654746),
 ('가장', 0.70710678118654746),
 ('물론', 0.57735026918962573),
 ('물어', 0.57735026918962573),
 ('인생 스토리', 0.57735026918962562),
 ('공유', 0.5),
 ('9회 녹화', 0.5),
 ('한명회 내린다', 0.45943360466390393),
 ('오후 30', 0.44721359549995793),
 ('오후', 0.44721359549995793),
 ('30', 0.44721359549995793),
 ('세계일주', 0.35355339059327373),
 ('세계일주 뭉쳐', 0.35355339059327373),
 ('jtbc 제공 jtbc 제공', 0.35355339059327373),
 ('jtbc 제공 jtbc', 0.35355339059327373),
 ('세계 일주', 0.35355339059327373),
 ('뭉쳐', 0.35355339059327373),
 ('일주', 0.35355339059327373),
 ('세계', 0.35355339059327373),
 ('제공', 0.35355339059327373),
 ('배경', 0.28867513459481281),
 ('기도', 0.28867513459481281),
 ('끌기도', 0.28867513459481281),
 ('스토리', 0.28867513459481281),
 ('스토리 기도', 0.28867513459481281),
 ('인생', 0.28867513459481281),


In [ ]:
vect = TfidfVectorizer(tokenizer=ct.phrases, ngram_range=(1, 1),
                    stop_words=stopwords)
count = vect.fit_transform(re.sub('\. ','.\n', data['mainText']).split('\n')).toarray().sum(axis = 0)
count

In [ ]:
def Search(x,li):
    if x in li:
        return li.index(x)
dx = list(map(lambda x: Search(x, vect.get_feature_names()), outlist))
dx = list(filter(lambda x: x!=None, dx))
y =  [1] * len(vect.get_feature_names())
for ix in dx:
    y[ix] = 1.5
count = count * y
idx = np.argsort(-count)
count = count[idx]
feature_name = np.array(vect.get_feature_names())[idx]
out = dict(zip(feature_name, count))
sorted(out.items(), key=lambda x: x[1], reverse=True)

In [26]:
vect = TfidfVectorizer(tokenizer=ct.phrases, ngram_range=(1, 1),
                    stop_words=stopwords)
vect.fit(morp1)
vect.fit(wsd1)
vect.fit(word1)
vect.fit(ne1)
vect.fit(srl1)
#vect.fit(outlist)
count = vect.transform(re.sub('\. ','.\n', data['mainText']).split('\n')).toarray()
count1 = count.sum(axis = 0)
def Search(x,li):
    if x in li:
        return li.index(x)
dx = list(map(lambda x: Search(x, vect.get_feature_names()), outlist))
dx = list(filter(lambda x: x!=None, dx))
y =  [1] * len(vect.get_feature_names())
for ix in dx:
    y[ix] = 5
p = count1 * y
#idx = np.argsort(-count1)
idx = np.argsort(-p)
count1 = count1[idx]
feature_name = np.array(vect.get_feature_names())[idx]
out = dict(zip(feature_name, count1))
sorted(out.items(), key=lambda x: x[1], reverse=True)

[('이름', 1.462073472281384),
 ('jtbc', 1.3410760885726434),
 ('한명회', 1.200812294088129),
 ('방송', 1.0777650988856593),
 ('출연자들', 0.91285085704502411),
 ('서로', 0.82937538578881664),
 ('특집', 0.63162804749320345),
 ('녹화', 0.59395102181825776),
 ('서로의 이름', 0.5625987354027806),
 ('이날', 0.54611399773526981),
 ('출연자', 0.45317762861951294),
 ('시간', 0.44065002381314938),
 ('지난 10월 10일', 0.42887908533317826),
 ('지난', 0.42887908533317826),
 ('10', 0.42887908533317826),
 ('10월 10일', 0.42887908533317826),
 ('최근', 0.39841145730442984),
 ('이름 특집', 0.36197879812939515),
 ('별난 이름 특집', 0.36197879812939515),
 ('별난', 0.36197879812939515),
 ('특집 명', 0.35025212164224351),
 ('조치', 0.35025212164224351),
 ('녹화 전', 0.35025212164224351),
 ('오후 9시 30분', 0.34428281216771534),
 ('오후', 0.34428281216771534),
 ('마지막 회', 0.34428281216771534),
 ('마지막', 0.34428281216771534),
 ('9시 30분', 0.34428281216771534),
 ('30', 0.34428281216771534),
 ('5일 오후 9시 30분', 0.34428281216771534),
 ('기자', 0.34107608857264343),
 ('이승', 0.341076

In [ ]:
d = vect.get_feature_names()

In [ ]:
len(d)

In [ ]:
count1[dx]

In [ ]:
count1[dx]

In [ ]:
f = np.array([1]*len(d))

In [ ]:
p[dx]

In [ ]:
print (count1[d])
count1[d] * np.array([1.25]*len(d)) 

In [ ]:
print (count1[d])

In [ ]:
`t.

In [ ]:
filte

In [ ]:
def Extract_Keywords(text, stopwords):
    etri = Run_ETRI_Analysis(stopwords, text)
    soy1 = RunLRNounExtractor(text)
    soy2 = RunWordExtractor(text)
    et0 = list(filter(lambda x: x[1] in ['NNG', 'NNP','NNG','SL','SH','NH'], etri[0]))
    et0 = list(map(lambda x: x[0], et0))
    et1 = list(filter(lambda x: x[1] in ['NNG', 'NNP','NNG','SL','SH','NH'], etri[1]))
    et1 = list(map(lambda x: x[0], et1))
    et2 = list(map(lambda x: x[0], etri[2]))
    et3 = list(map(lambda x: x[0], etri[3]))
    et4 = list(filter(lambda x: x[1] in ['ARG0','ARG1','ARG2','ARG3','ARG4'], etri[4]))
    et4 = list(map(lambda x: x[0], et4))
    et5 = list(filter(lambda x: x[1] in ['NP','NP_SBJ','NP_OBJ','NP_AJT','VNP'], etri[5]))
    et5 = list(map(lambda x: x[0], et5))
    mecabout = list(filter(lambda x: x[1] in ['NNG','NNB','NNP'], mecab.pos(text)))
    ctout = list(filter(lambda x: x[1] == 'Noun' , ct.pos(text)))
    otout = list(filter(lambda x: x[1] == 'Noun' , ot.pos(text)))
    mcout = list(map(lambda x: x[0], mecabout))
    ctout = list(map(lambda x: x[0], ctout))
    otout = list(map(lambda x: x[0], otout))
    out = list(soy1.keys())+list(soy2.keys())+ctout+otout+mcout+et3+et4+et0+et1+et2+et5
    vect = TfidfVectorizer().fit(out)
    y = [list(soy1.keys()), list(soy2.keys()),et3, et4, ctout, otout, mcout,et0, et1, et2, et5]
    y = list(filter(lambda x: len(x) !=0, y))
    outdict = dict()
    for i in y:
        count = vect.transform(i).toarray().sum(axis = 0)
        idx = np.argsort(-count)
        count = count[idx]
        feature_name = np.array(vect.get_feature_names())[idx]
        out = dict(zip(feature_name, count))
        for ii in out:
            if not ii in outdict:
                outdict[ii] = out[ii]
            else:
                outdict[ii] +=out[ii]
    x = sorted(outdict.items(), key = itemgetter(1), reverse=True)[:5]
    output = list(map(lambda x: x[0], x))
    return output


In [ ]:
def Extract_Keyword_from_news_bigdata(path):
    from glob import glob
    fileList = glob(path+'*.csv')
    listIs =list()
    for i in fileList:
        df = pd.read_csv(i,engine='python')
        f = list(filter(lambda x: x[1] in ['중요키워드','키워드','keword','토픽키워드','토픽 키워드'], enumerate(df.columns.values)))
        if not len(f) == 0:
            listIs += list(df[df.columns[f[0][0]]].values)
    outlist = []
    for ix in list(filter(lambda x: type(x)!=float, listIs)):
        outlist += ix.split(',')
    outlist1 = list(map(lambda x: ''.join(x.split('_')), outlist))
    outlist2 = list(map(lambda x: ' '.join(x.split('_')), outlist))
    return outlist1+outlist2

In [ ]:
for data in useCollection.find({'site':'daum'}):
    if len(data['mainText'].strip()) !=0:
        if data['keywords'] !='NaN':
            ct.add_dictionary(list(map(lambda x: x.split(' · ')[-1], data['keywords'])), 'Noun')
        y = Extract_Keywords2(data['mainText'],stopwords)
        z = TextRank(data['mainText'])
        p = RunLRNounExtractor(data['mainText'])
        q = RunWordExtractor(data['mainText'])
        #print (y)
        print (data['keywords'])
        print (z.keywords())
        #print (p.keys())
        #print (q.keys())
        print ()
        break

# sentiment analysis

In [ ]:
import os
import sys

In [ ]:
def read_data(filename):
    with codecs.open(filename, encoding = 'utf-8', mode='r') as f:
        data